In [1]:
'''This script loads pre-trained word embeddings (GloVe embeddings)
into a frozen Keras Embedding layer, and uses it to
train a text classification model on the 20 Newsgroup dataset
(classification of newsgroup messages into 20 different categories).
GloVe embedding data can be found at:
http://nlp.stanford.edu/data/glove.6B.zip
(source page: http://nlp.stanford.edu/projects/glove/)
20 Newsgroup data can be found at:
http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html
'''

from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

Using TensorFlow backend.


In [2]:
BASE_DIR = ''
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
TEXT_DATA_DIR = os.path.join(BASE_DIR, '20_newsgroup')
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [3]:
# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')
embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
        
print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [4]:
embeddings_index['hello']

array([ 0.26688001,  0.39631999,  0.61690003, -0.77451003, -0.1039    ,
        0.26697001,  0.27880001,  0.30992001,  0.0054685 , -0.085256  ,
        0.73602003, -0.098432  ,  0.54790002, -0.030305  ,  0.33478999,
        0.14094   , -0.0070003 ,  0.32569   ,  0.22902   ,  0.46557   ,
       -0.19531   ,  0.37491   , -0.71390003, -0.51775002,  0.77038997,
        1.08809996, -0.66011   , -0.16234   ,  0.91189998,  0.21046001,
        0.047494  ,  1.00189996,  1.11329997,  0.70094001, -0.08696   ,
        0.47571   ,  0.1636    , -0.44468999,  0.44690001, -0.93817002,
        0.013101  ,  0.085964  , -0.67456001,  0.49662   , -0.037827  ,
       -0.11038   , -0.28612   ,  0.074606  , -0.31527001, -0.093774  ,
       -0.57068998,  0.66864997,  0.45307001, -0.34154001, -0.7166    ,
       -0.75273001,  0.075212  ,  0.57902998, -0.1191    , -0.11379   ,
       -0.10026   ,  0.71341002, -1.15740001, -0.74026   ,  0.40452   ,
        0.18023001,  0.21449   ,  0.37638   ,  0.11239   , -0.53

In [5]:
# second, prepare text samples and their labels
print('Processing text dataset')

texts = []  # list of text samples 所有文本的列表
labels_index = {}  # dictionary mapping label name to numeric id 类名与序号的映射字典
labels = []  # list of label ids 每个文本对应的类的编号
for name in sorted(os.listdir(TEXT_DATA_DIR)): # 每个类的名字
    path = os.path.join(TEXT_DATA_DIR, name) #每个类所在文件夹的路径
    if os.path.isdir(path): 
        label_id = len(labels_index) 
        labels_index[name] = label_id # 类名与序号的映射字典
        for fname in sorted(os.listdir(path)): # fname 为每个类下的每个文件的名字
            if fname.isdigit(): 
                fpath = os.path.join(path, fname) #获得文件的路径
                args = {} if sys.version_info < (3,) else {'encoding': 'latin-1'}
                with open(fpath, **args) as f:
                    t = f.read()
                    i = t.find('\n\n')  # skip header
                    if 0 < i:
                        t = t[i:] # 去掉头部无用文字，获取正文
                    texts.append(t)
                labels.append(label_id)

print('Found %s texts.' % len(texts))

Processing text dataset
Found 19997 texts.


In [6]:
labels_index['alt.atheism']

0

In [7]:
labels_index['comp.graphics']

1

In [8]:
len(labels_index) # 一共20个类

20

In [20]:
len(labels) #一共19997篇文本

19997

In [44]:
len(texts[0])

11521

In [24]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)#分词器将被限制为待处理数据集中最常见的num_words个单词
tokenizer.fit_on_texts(texts)

In [34]:
type(texts[0])

str

In [27]:
sequences = tokenizer.texts_to_sequences(texts) #将每个文本序列化，每个数字代表在词典中的位置

In [42]:
sequences[0]

[1237,
 273,
 1213,
 1439,
 1071,
 1213,
 1237,
 273,
 1439,
 192,
 2515,
 348,
 2964,
 779,
 332,
 28,
 45,
 1628,
 1439,
 2516,
 3,
 1628,
 2144,
 780,
 937,
 29,
 441,
 2770,
 8854,
 4601,
 7969,
 11979,
 5,
 12806,
 75,
 1628,
 19,
 229,
 29,
 1,
 937,
 29,
 441,
 2770,
 6,
 1,
 118,
 558,
 2,
 90,
 106,
 482,
 3979,
 6602,
 5375,
 1871,
 12260,
 1632,
 17687,
 1828,
 5101,
 1828,
 5101,
 788,
 1,
 8854,
 4601,
 96,
 4,
 4601,
 5455,
 64,
 1,
 751,
 563,
 1716,
 15,
 71,
 844,
 24,
 20,
 1971,
 5,
 1,
 389,
 8854,
 744,
 1023,
 1,
 7762,
 1300,
 2912,
 4601,
 8,
 73,
 1698,
 6,
 1,
 118,
 558,
 2,
 1828,
 5101,
 16500,
 13447,
 73,
 1261,
 10982,
 170,
 66,
 6,
 1,
 869,
 2235,
 2544,
 534,
 34,
 79,
 8854,
 4601,
 29,
 6603,
 3388,
 264,
 1505,
 535,
 49,
 12,
 343,
 66,
 60,
 155,
 2,
 6603,
 1043,
 1,
 427,
 8,
 73,
 1698,
 618,
 4601,
 417,
 1628,
 632,
 11716,
 4602,
 814,
 1628,
 691,
 3,
 1,
 467,
 2163,
 3,
 2266,
 7491,
 5,
 48,
 15,
 40,
 135,
 378,
 8,
 1,
 467,
 6359,
 

In [35]:
word_index = tokenizer.word_index #返回字典类型，将单词（字符串）映射为它们的排名或者索引。
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 174074 unique tokens.
Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)


In [55]:
word_index['am']

112

In [45]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]


In [46]:
num_validation_samples

3999

In [47]:
x_train.shape

(15998, 1000)

In [48]:
x_val.shape

(3999, 1000)

In [50]:
print(MAX_NUM_WORDS)
print(len(word_index) )

20000
174074


In [56]:
# prepare embedding matrix
print('Preparing embedding matrix.')

num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

Preparing embedding matrix.


In [57]:
# train a 1D convnet with global maxpooling
print('Training model.')
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
model.summary()

Training model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         2000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 35, 128)           82048

In [58]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 10s - loss: 2.6226 - acc: 0.1259 - val_loss: 2.2455 - val_acc: 0.2198
Epoch 2/10
15998/15998 [==============================] - 8s - loss: 1.9831 - acc: 0.3085 - val_loss: 1.7260 - val_acc: 0.3831
Epoch 3/10
15998/15998 [==============================] - 8s - loss: 1.5362 - acc: 0.4574 - val_loss: 1.5859 - val_acc: 0.4401
Epoch 4/10
15998/15998 [==============================] - 8s - loss: 1.2780 - acc: 0.5623 - val_loss: 1.1375 - val_acc: 0.6109
Epoch 5/10
15998/15998 [==============================] - 8s - loss: 1.0876 - acc: 0.6271 - val_loss: 1.0764 - val_acc: 0.6229
Epoch 6/10
15998/15998 [==============================] - 8s - loss: 0.9450 - acc: 0.6711 - val_loss: 1.0140 - val_acc: 0.6532
Epoch 7/10
15998/15998 [==============================] - 8s - loss: 0.8203 - acc: 0.7148 - val_loss: 0.9496 - val_acc: 0.6792
Epoch 8/10
15998/15998 [==============================] - 8s 